In [91]:
import geopandas as gpd
import turfpy
from geojson import Point, Feature, FeatureCollection
from ipyleaflet import Map, GeoJSON, LayersControl, Marker
import folium
import numpy as np
import ee
import geemap
from turfpy.measurement import nearest_point, distance

In [ ]:
map = geem

In [2]:
geojson_path = "E:/Satyukt/paired_pilot/data/shp/VILLAGE.shp"

gdf = gpd.read_file(geojson_path)

## Calculate the Centroid
gdf['centroid'] = [x.centroid for x in gdf.geometry]

#For meters
gdf = gdf.to_crs(3763)

C:\Users\Asus\miniconda3\envs\gis\lib\site-packages\pandas\core\dtypes\cast.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [3]:
## Initialize the Map
m = Map(center = [18.42,77.35], zoom = 12)

# Convert the centroids in gdf to a FeatureCollection
feature_collection = FeatureCollection([Feature(geometry=Point(list(gdf['centroid'][i].coords)[0])) for i in range(len(gdf))])

In [111]:
def getNearestPointDistance(pt,dist):
    '''
    Returns the Nearest Point in the Feature collection
    It takes two arguments:
        pt >> shapely.coordinates.sequence
        dist >> Its the boolean value which if set to true, 
        Returns the distance in meters to that point
    '''
    global feature_collection
    point = Point(pt)
    if dist == True:
        return (distance(point,nearest_point(point, feature_collection)))
    
    return nearest_point(point, feature_collection)


def bufferPoint(pt):
    '''
    Takes the Shapely.geometry.point object and returns the buffer of 50 meters
    '''
    return gpd.GeoSeries(pt).buffer(50)



def pointsInBuffer(buff):
    '''
    Takes the Buffer of a point and returns all the points which lies inside of the Polygon
    It returns the GeoDataFrame of the Points.
    '''
    global gdf
    return gdf.iloc[[buffer.contains(gdf['centroid'][i])[0] for i in range(len(gdf))]] 


def convShapelyGeometry(coords):
    '''
    Convert the Shapely.geometry.Point object to ee.Geometry.Point object
    '''
    return ee.Geometry.Point(list(coords.coords)[0])


def getNDVI(coords):
    '''
    Calculate the NDVI value of the first image for the ee.Geometry.Point object
    Returns NDVI value
    '''
    coords = convShapelyGeometry(coords)
    
    s2_collection = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(coords).first()
    
    ndvi = image.normalizedDifference(['B8', 'B4'])
    
    # Get the value of NDVI at the point of interest
    point_ndvi = ndvi.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=ee_point,
        scale=10
    )

    ndvi_value = point_ndvi.get('nd').getInfo()
    return ndvi_value
    

0   -0.00521
1   -0.00521
2   -0.00521
3   -0.00521
4   -0.00521
5   -0.00521
Name: centroid, dtype: float64